<a href="https://colab.research.google.com/github/dastone16/utils_python/blob/main/high_low_days.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def high_low_days(data,resample,Output_name,variable,days=10):
  #input already averaged data by day.  

  #originally used for only mA, switched to any variable, but temporary variables inside function still refer to mA.
  data_Day = data.resample(resample).mean()
  data_Day = data_Day.dropna(how='all')
  data_Day = data_Day.dropna(how='all',axis='columns')
  data_Day = data_Day.iloc[(-np.abs(data_Day[variable].values)).argsort()]
  data_High20_mA_Days = data_Day.iloc[0:days,:]
  data_Low20_mA_Days = data_Day.iloc[-days:-1,:]
  #labeled mA because originally only written for mA.  Added ability to input vs any variable.

  data_High_mean = data_High20_mA_Days.resample('10AS').mean() #resample by year to average
  data_High_mean = data_High_mean.reset_index(drop=True).transpose()
  data_Low_mean = data_Low20_mA_Days.resample('10AS').mean()
  data_Low_mean = data_Low_mean.reset_index(drop=True).transpose()
  data_Range_max = data_Day.resample('10AS').max()  # by year to get entire range
  data_Range_max = data_Range_max.reset_index(drop=True).transpose()
  data_Range_min = data_Day.resample('10AS').min()
  data_Range_min = data_Range_min.reset_index(drop=True).transpose()
  data_High_mean.rename(columns={0:'Avg: '+str(days)+' Highest Days'}, inplace=True)
  data_Low_mean.rename(columns={0:'Avg: '+str(days)+' Lowest Days'}, inplace=True)
  data_Range_max.rename(columns={0:'Max'}, inplace=True)
  data_Range_min.rename(columns={0:'Min'}, inplace=True)

  data_Summary = data_Low_mean
  data_Summary['Avg: '+str(days)+' Highest Days'] = data_High_mean['Avg: '+str(days)+' Highest Days']
  data_Summary['Difference'] = data_High_mean['Avg: '+str(days)+' Highest Days']-data_Low_mean['Avg: '+str(days)+' Lowest Days']
  data_Summary['Max'] = data_Range_max['Max']
  data_Summary['Min'] = data_Range_min['Min']
  data_Summary['Range'] = data_Summary['Max'] - data_Summary['Min']
  data_Summary['% of Range'] = data_Summary['Difference']/data_Summary['Range']*100
  data_Summary = data_Summary.iloc[(-np.abs(data_Summary['% of Range'].values)).argsort()]
  data_Summary.to_csv(save_path + Output_name + variable + str(days)+'days.High.Low.diff.csv')
  return data_Summary